<a href="https://colab.research.google.com/github/yleessam/2501_dl_env_colab/blob/main/10_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

건물 지붕 위의 센서에서 측정한 기압, 습도 등 매시간 기록된 시계열 데이터를 바탕으로 24시간 후의 온도를 예측하는 것입니다.  시계열 데이터가 우리가 지금까지 다룬 데이터셋들과는 근본적으로 다르다는 점을 잘 보여주는 사례가 온도 예측입니다.    순환 신경망 RNN이 이런 종류의 문제에 뛰어나다는 것을 확인 할 수 있습니다.

이 데이터는 독일 예나(Jena)시에 있는 막스 플랑크 생물지구화학연구소(Max Planck Institute for Biogeochemistry)의 기상 관측소에서 수집되었습니다.

이 데이터셋은 온도, 기압, 습도, 풍향 등 14개의 관측치가 10분 간격으로 수년간 기록되어 있습니다.

원본 데이터는 2003년부터 기록되어 있지만, 이 예제에서는 2009~2016년 사이의 데이터만 사용합니다.

먼저 데이터를 내려받고 살펴보겠습니다.

https://www.kaggle.com/datasets/stytch16/jena-climate-2009-2016

In [ ]:
!wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
!unzip jena_climate_2009_2016.csv.zip

In [ ]:
import os
fname = os.path.join("jena_climate_2009_2016.csv")

with open(fname) as f:
    data = f.read()

lines = data.split("\n")
header = lines[0].split(",")
lines = lines[1:]
print(header)
print(len(lines))

출력된 데이터는 총 420,551줄이며, 각 줄은 하나의 타임스텝으로 날짜와 14개의 날씨 정보를 포함하고 있습니다.  헤더는 다음과 같습니다

- Date Time (날짜 시간)
- p (기압, mbar)
- T (온도, degC)
- Tpot (K)
- Tdew (이슬점, degC)
- rh (상대습도, %)
- VPmax (mbar)
- VPact (mbar)
- VPdef (mbar)
- sh (비습도, g/kg)
- H2OC (mmol/mol)
- rho (밀도, g/m³)
- wv (풍속, m/s)
- max. wv (최대 풍속, m/s)
- wd (풍향, deg)

420,551개의 데이터를 넘파이 배열로 변환합니다.

온도(섭씨)는 하나의 배열로, 나머지 데이터는 별도의 배열로 만듭니다.

두 번째 배열은 미래 온도 예측을 위한 특성으로 사용됩니다.

'Date Time' 열은 제외합니다.

In [ ]:
import numpy as np
temperature = np.zeros((len(lines),))
raw_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(",")[1:]]
    temperature[i] = values[1]
    raw_data[i, :] = values[:]

시간에 따른 온도(섭씨) 변화를 보여주는 그래프입니다.

이 그래프를 통해 8년간의 데이터에서 매년 반복되는 온도의 주기성을 명확하게 확인할 수 있습니다.

In [ ]:
from matplotlib import pyplot as plt
plt.plot(range(len(temperature)), temperature)
plt.show()

데이터는 10분 간격으로 기록되어 하루에 144개의 데이터 포인트가 수집됩니다.

각 분할에 사용할 샘플 수 계산하기

데이터 정규화

훈련, 검증, 테스트 데이터셋 만들기

상식적인 기준 모델의 MAE 계산하기

밀집 연결 모델 훈련하고 평가하기

In [ ]:
import matplotlib.pyplot as plt
loss = history.history["mae"]
val_loss = history.history["val_mae"]
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, "bo", label="Training MAE")
plt.plot(epochs, val_loss, "b", label="Validation MAE")
plt.title("Training and validation MAE")
plt.legend()
plt.show()

1D 합성곱 모델

간단한 LSTM 기반 모델

넘파이로 구현한 간단한 RNN

어떤 길이의 시퀀스도 처리할 수 있는 RNN 층

마지막 출력 스텝만 반환하는 RNN 층

전체 출력 시퀀스를 반환하는 RNN 층

스태킹(stacking) RNN 층

과대적합을 감소하기 위해 순환 드롭아웃 사용하기

드롭아웃 규제와 스태킹을 적용한 GRU 모델을 훈련하고 평가하기

양방향 RNN 사용하기